<a href="https://colab.research.google.com/github/cyberneel/ImageSecretEmbed/blob/main/ImageEmbedSecret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [14]:
!pip install pillow numpy

# Main Code

In [15]:
from PIL import Image
import numpy as np
import io
from google.colab import files

def ToBinary(text):
    binary = ''.join(format(ord(char), '08b') for char in text)
    return binary

def ToText(binary):
    characters = [binary[i:i+8] for i in range(0, len(binary), 8)]
    textForm = ''.join([chr(int(char, 2)) for char in characters])
    return textForm

# Used to test proper conversion
print(ToBinary("Hello"))
print(ToText(ToBinary("Hello")))

0100100001100101011011000110110001101111
Hello


In [16]:
def EncodeIntoImage(imagePath, text):
    image = Image.open(imagePath)
    textBinary = ToBinary(text) + '1111111111111110'  # This will help find the end
    binTextLen = len(textBinary)

    # Turn image into data with numpy
    imageData = np.array(image)
    height, width, colorChannels = imageData.shape # simialr to pytorch

    # Now loop through the binary
    dataIndex = 0
    for y in range(height):
      for x in range(width):
        for c in range(3): # represents rgb
          if (dataIndex < binTextLen):
            imageData[y, x, c] = int(textBinary[dataIndex])
            dataIndex += 1

    TextEncodedImage = Image.fromarray(imageData)
    return TextEncodedImage


In [25]:
def DecodeFromImage(imagePath):
  image = Image.open(imagePath)

  # Turn image. into data
  imageData = np.array(image)
  height, width, colorChannels = imageData.shape
  BinaryFromImage = ""

  # pixels loop
  for y in range(height):
    for x in range(width):
      for c in range(3):
        if imageData[y, x, c] == 1 or imageData[y, x, c] == 0:
          print(imageData[y,x,c])
          BinaryFromImage += str(imageData[y, x, c])

  return ToText(BinaryFromImage)

In [26]:
# past image path from sidebar and text, then encode
imagePath = "/content/lp_image.jpeg"

text = input("Enter the secret to encode into the image: ")
encoded_img = EncodeIntoImage(imagePath, text)
encoded_img.save('secret.png')

DecodeFromImage("/content/secret.png")

Streaming output truncated to the last 5000 lines.
1
0
0
1
1
1
1
1
0
1
1
1
0
0
0
1
0
0
1
1
1
1
1
1
1
1
1
0
0
1
1
0
0
0
1
0
0
0
0
1
1
1
1
1
0
1
1
1
0
0
0
1
0
0
0
0
1
1
1
1
1
1
1
1
1
0
1
0
0
0
1
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
1
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
0
0
0
1
0
0
0
1
0
0
0
1
1
1
1
1
0
1
1
1
0
0
0
0
0
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
1
0
0
0
0
0
1
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
0
1
0
1
1
0
0
0
0
0
1
1
1
1
1
0
1
1
1
0
1
1
1
0
1
1
1
0
0
1
1
1
0
0
0
0
1
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1
0
1
1
1
1
1
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
1
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
1
1
1
0
0
0
0
1
1
1
0
0
0
1
0
0
1
1
1
0
0
0
0
0
1
0
1
1
0
0
1
1
1
1
0
0
0
1
1
1
1

'Helloÿþ@\x01&p\x8f\x0c<a\x82\x18ò \x08\x10ÿý?þQâ\x80\x02\x12\x08\x8c¨\x80¤\x00f\x00\x01\t~_â8þ\x04\x07ãÁÇ\x00\x0e\x00\x00\x7f\x808nø\x00±ôB\x01Ò \t¤\x00\x9e\x00\x13ª9Ðsâçñsõyê\x03ê\x01\xa0\x06\x00\x0c\x00\x1c\x18P\x9e\x1f\x0e>°\x03\x08¡\x04]Ü`\x00\x00ÿ\x0fð\x14ÿ¸«þ¯ó\x8c¯ÿ\x89ÿì\x1e\x7fò\x8fÇø¿<\tw+\x7f¾\rÿ¡ý?ñáÿ\x18C\x87\x00À\x81\x8c|\x00\x8cò\x07\x90>ððü\x0fÿÃý\x80þ@\x821!äsøðÿÀ\x088\x00à\x15T@\x08\x02\x10\x9e\x7f\x87\x02\x90??8\x00yð\x0cyÀ\x1fü¨U\x00\x07\x90\x7fÿÿÃü°ÿðø?î\x83ß2\x06Uz\xa0ð&\x82¨~\x01ÿüD@ª\x8fßÿ!"\x00\x0fø\x02!Uÿ°,\x80%\\C \x07"\x8f\x80ò*\x07ø\xa0>#\x03\x81\x81ð`~\x10xpCáô\x1e}\x13\x7fá?ü&ü\x0eàyÀø\x1f!ò\x9f\x13à}\x1fû\x0f\x07\x81Çÿ\xa0çÿ\x8cwÿÂ?ÿ\x92?ò\x8fò9\x84æ/¢\x17ñ\x85üBÿ\x9a\x8fù¤\x7fá\x1fð\x8føV\x1eñé{ãß=þò÷ñËÞ\x07\x03_ùà!ïþ|D#ßÿï?\x19÷ÿÿù\x957ÿÿö\x86ÿÿñßÿø\x9fÿä>\x06\x91á\x9c<qð\x87\x8dGÄÀáÇà\x8f\x84~\x82?ÅGÃÀ<\x01\x07À\x0f½¨\x0f\x9f\x10_ø\x17ü\x17ýEÿå/ÿþ%ÿÿà\x10\x7fÿÿB\x0fÿÿÀGÿøCÿþ8\x87ÿø\x85\x7føÅ\x7fé\x82ÿÕZ?ú*Ñÿ\nÑø-\x1f\x82Ðø\t\x0f\x80\x90ô\x82Cÿ\nu\x1f